# Homework 4 (Sun Yi)
Runner

In [1]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import os
import sqlite3

/Users/sunyi/opt/anaconda3/lib/python3.9/site-packages/papermill/iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [31]:
output = pd.read_sql(f'''
SELECT 
    token, max(ts)
FROM ohlc 
group by token
order by sum(volumeUSD) desc
''', connection_string)
output

,token,max(ts)
0,BTC,2021-12-15 00:00:00
1,ETH,2021-12-15 00:00:00
2,SOL,2021-12-15 00:00:00
3,ADA,2021-12-15 00:00:00
4,AVAX,2021-12-15 00:00:00
5,DOT,2021-12-15 00:00:00
6,USDT,2021-12-15 00:00:00
7,ATOM,2021-12-15 00:00:00
8,CRV,2021-12-15 00:00:00
9,AAVE,2021-12-15 00:00:00


In [24]:
start_date = pd.to_datetime('2021-12-15 00:00:00') - timedelta(days=14)
start_date

Timestamp('2021-12-01 00:00:00')

In [32]:
run_dates = [(start_date + timedelta(days=7 * x)) for x in range(2)]
tokens = output.token.to_list()
connection_string = 'sqlite:///data/data.db'
events_db = 'crypto_events_database'

In [33]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [35]:
for run_date in run_dates:
    for token in tokens:
        print(f'running for {token} for date {run_date} to {run_date + timedelta(days=7)}', end='\r')
        res = pm.execute_notebook(
            'hw_4_yi_sun_template.ipynb',
            f'{base_output_folder}/market_analysis_{token}_{run_date}.ipynb',
            parameters = {
                'from_date': f'{run_date}',
                'to_date': f'{run_date + timedelta(days=7)}',
                'token': token,
                'connection_string': connection_string,
                'events_db': events_db
            }
        )

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

Executing:   0%|          | 0/12 [00:00<?, ?cell/s]

In [40]:
nbs = sb.read_notebooks('outputs')

In [71]:
summary = pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'avg hourly return' : scrap['prices'].data['return'].mean(),
        'avg volatility' : scrap['prices'].data['volatility'].mean(),
        'max drawdown' : scrap['prices'].data['maxDrawdown'].min(),
        'last close - first close' : scrap['prices'].data['close'].sort_index().iloc[-1] - 
                                     scrap['prices'].data['close'].sort_index().iloc[0]})
    for book, scrap in nbs.notebook_scraps.items()
])
summary

,token,from_date,to_date,avg hourly return,avg volatility,max drawdown,last close - first close
0,AAVE,2021-12-01 00:00:00,2021-12-08 00:00:00,-1.911719e-03,0.242483,-0.365927,-73.0010
1,AAVE,2021-12-08 00:00:00,2021-12-15 00:00:00,-5.628913e-04,0.190437,-0.214169,-18.3390
2,ADA,2021-12-01 00:00:00,2021-12-08 00:00:00,-6.500425e-04,0.259161,-0.270810,-0.1819
3,ADA,2021-12-08 00:00:00,2021-12-15 00:00:00,-4.394820e-04,0.192457,-0.143597,-0.1095
4,ATOM,2021-12-01 00:00:00,2021-12-08 00:00:00,-6.660243e-04,0.374788,-0.318250,-3.6450
5,ATOM,2021-12-08 00:00:00,2021-12-15 00:00:00,-5.848282e-04,0.236202,-0.169664,-2.5920
6,AVAX,2021-12-01 00:00:00,2021-12-08 00:00:00,-1.707432e-03,0.336614,-0.385790,-32.8200
7,AVAX,2021-12-08 00:00:00,2021-12-15 00:00:00,-1.469500e-04,0.232938,-0.179823,-3.5700
8,BTC,2021-12-01 00:00:00,2021-12-08 00:00:00,-7.111839e-04,0.183768,-0.205844,-6702.1000
9,BTC,2021-12-08 00:00:00,2021-12-15 00:00:00,-2.573223e-04,0.133933,-0.081655,-2336.4900
